In [0]:
!nvidia-smi

Sat May  2 08:15:23 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P0    45W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
#@title Import
# first run you need to do the following
# upload {train, test, sample_submission}.csv to ./
# !pip install transformers
# nltk.download('stopwords')

import torch
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler,SequentialSampler

import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup

import pandas as pd
import numpy as np
import nltk
import re
import time
import datetime
import random
import matplotlib.pyplot as plt
% matplotlib inline
import seaborn as sns

# https://mccormickml.com/2019/07/22/BERT-fine-tuning/

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
#@title GPU

device_name = tf.test.gpu_device_name()
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

if torch.cuda.is_available():    

    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

Found GPU at: /device:GPU:0
There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [0]:
#@title Helper Functions

def clean_text(text_arr):
    '''filter out stopwords or words with non-alphabetical characters
    
    Args:
        text_arr: list of strings, each representing sentences
    
    Returns:
        2d list of strings, each representing words
    '''
    stop_words = set(nltk.corpus.stopwords.words('english'))
    tokenizer = nltk.tokenize.TweetTokenizer()
    clean = []
    
    for sent in text_arr:
        sent = re.sub(r"[#@_]", " ", sent)
        tokens = tokenizer.tokenize(sent)
        list_sent = []
        for word in tokens:
            word = word.strip('#').lower()
            if word.isalpha() and not word in stop_words:
                list_sent.append(word)
        clean.append(list_sent)
    return clean

def flat_accuracy(preds, labels):
    '''calculate the accuracy of our predictions vs labels
    '''
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def tfpn(logit, label, threshold=0.5, beta=2):
    '''takes two torch tensors and return 4 torch tenors
    '''
    softmax = torch.nn.Softmax(dim=1)
    sm = softmax(logit)
    pred = torch.argmax(sm, dim=1).cpu()
    label = (label > threshold).cpu()

    TP = (pred & label).sum()
    TN = ((~pred) & (~label)).sum()
    FP = (pred & (~label)).sum()
    FN = ((~pred) & label).sum()

    return TP, TN, FP, FN

def f1(tfpn_dict):
    '''takes a dict of torch tensors and returns a torch tensor
    '''
    TP, TN, FP, FN = tfpn_dict['TP'], tfpn_dict['TN'], tfpn_dict['FP'], tfpn_dict['FN']
    precision = torch.true_divide(TP, TP + FP + 1e-12)
    recall = torch.true_divide(TP, TP + FN + 1e-12)
    f1 = 2 * (precision * recall) / (precision + recall + 1e-12)
    return f1

In [0]:
## Parameters

maxlen = 15
train_val_split = 0.25
batch_size = 32
random_seed = 84

In [0]:
#@title Load Data

# tokenize

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

df_train = pd.read_csv('train.csv')
x_train_str = clean_text([each for each in df_train['text']])
y_train = [each for each in df_train['target']]

x_train_ids = []
x_train_att_mask = []

for sent in x_train_str:
    sent = [' '] if len(sent) == 0 else sent
    encoded_dict = tokenizer.encode_plus(sent, 
                                         max_length=maxlen, 
                                         pad_to_max_length=True, 
                                         return_attention_mask=True, 
                                         return_tensors='pt')
    x_train_ids.append(encoded_dict['input_ids'])
    x_train_att_mask.append(encoded_dict['attention_mask'])

x_train_ids = torch.cat(x_train_ids, dim=0)
x_train_att_mask = torch.cat(x_train_att_mask, dim=0)
y_train = torch.tensor(y_train)

# split training for validation

dataset = TensorDataset(x_train_ids, x_train_att_mask, y_train)

val_size = int((train_val_split) * len(dataset))
train_size = len(dataset) - val_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print(train_size, 'training samples')
print(val_size, 'validation samples')

# create dataloaders

train_dataloader = DataLoader(train_dataset,
                              sampler = RandomSampler(train_dataset),
                              batch_size = batch_size)

# for validation the order doesn't matter, so just read them in order
val_dataloader = DataLoader(val_dataset,
                              sampler = SequentialSampler(val_dataset),
                              batch_size = batch_size)

# testing data

df_test = pd.read_csv('test.csv')
x_test_str = clean_text([each for each in df_test['text']])

x_test_ids = []
x_test_att_mask = []

for sent in x_test_str:
    sent = [' '] if len(sent) == 0 else sent
    encoded_dict = tokenizer.encode_plus(sent, 
                                         max_length=maxlen, 
                                         pad_to_max_length=True, 
                                         return_attention_mask=True, 
                                         return_tensors='pt')
    x_test_ids.append(encoded_dict['input_ids'])
    x_test_att_mask.append(encoded_dict['attention_mask'])

x_test_ids = torch.cat(x_test_ids, dim=0)
x_test_att_mask = torch.cat(x_test_att_mask, dim=0)

prediction_dataset = TensorDataset(x_test_ids, x_test_att_mask)
prediction_dataloader = DataLoader(prediction_dataset,
                                   sampler=SequentialSampler(prediction_dataset),
                                   batch_size=batch_size)

print(len(x_test_str), 'testing inputs')

print('\nbatch size set to', batch_size)

5710 training samples
1903 validation samples
3263 testing inputs

batch size set to 32


In [0]:
## Hyper-parameters

epochs = 5
lr = 2e-5
eps = 1e-8
num_labels = 2

In [0]:
#@title Train
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',
                                                      num_labels=num_labels,
                                                      output_attentions = False,
                                                      output_hidden_states =False)
model.cuda()

optimizer = AdamW(model.parameters(),
                  lr = lr,
                  eps = eps)

scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=len(train_dataloader) * epochs)

random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        loss, logits = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
        


        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # F1
    tfpn_dict = {'TP':0, 'TN':0, 'FP':0, 'FN':0}

    # Evaluate data for one epoch
    for batch in val_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            (loss, logits) = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)

        TP, TN, FP, FN = tfpn(torch.from_numpy(logits), b_labels)
        tfpn_dict['TP'] += TP
        tfpn_dict['TN'] += TN
        tfpn_dict['FP'] += FP
        tfpn_dict['FN'] += FN

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(val_dataloader)

    # calculate f1 score for this val epoch
    f1_score = f1(tfpn_dict).item()

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(val_dataloader)

    print('Validation - Loss: {:.3f} | Accuracy: {:.3f} | BCE: {:.3f} | F1: {:.3f}'.format(avg_val_loss, avg_val_accuracy, avg_BCE_loss, f1_score))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Train Loss': avg_train_loss,
            'Val Loss': avg_val_loss,
            'Val Accuracy': avg_val_accuracy,
            'Val F1': f1_score
        }
    )

print("")
print("Training complete!")

In [0]:
#@title Predict
model.eval()

predictions = []

for batch in prediction_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)

    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask = batch

    # Telling the model not to compute or store gradients, saving memory and 
    # speeding up prediction
    with torch.no_grad():
        # Forward pass, calculate logit predictions
        outputs = model(b_input_ids, token_type_ids=None, 
                        attention_mask=b_input_mask)

    logits = outputs[0]

    # Move logits and labels to CPU
    logits = logits.detach().cpu()

    # logits to label
    softmax = torch.nn.Softmax(dim=1)
    sm = softmax(logits)
    pred = torch.argmax(sm, dim=1)

    # Store predictions and true labels
    predictions += pred.tolist()

sample_submission = pd.read_csv("sample_submission.csv")
sample_submission["target"] = predictions
sample_submission.to_csv("submission.csv", index=False)

print('submission.csv generated')

submission.csv generated
